In [1]:
import os
os.chdir("../")
%pwd

'/Users/tapankheni/Data_Science/Data Science Projects/Predictive_Maintenance_JetEngine'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    x_train_data_path: Path
    y_train_data_path: Path
    model_name: str

In [9]:
from PredictiveMaintenance.constants import CONFIG_YAML_FILE_PATH, PARAMS_YAML_FILE_PATH, SCHEMA_YAML_FILE_PATH
from PredictiveMaintenance.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_YAML_FILE_PATH,
        params_filepath = PARAMS_YAML_FILE_PATH,
        schema_filepath = SCHEMA_YAML_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.CatBoostRegressor

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            x_train_data_path=config.x_train_data_path,
            y_train_data_path=config.y_train_data_path,
            model_name=config.model_name,
        )

        return model_trainer_config

In [11]:
import pandas as pd
import joblib
from PredictiveMaintenance import logger
from catboost import CatBoostRegressor

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        X_train = pd.read_csv(self.config.x_train_data_path)
        y_train = pd.read_csv(self.config.y_train_data_path)

        catboost_regressor = CatBoostRegressor()
        catboost_regressor.fit(X_train, y_train)

        joblib.dump(catboost_regressor, os.path.join(self.config.root_dir, self.config.model_name))

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    obj = ModelTrainer(config=model_trainer_config)
    obj.train()
except Exception as e:
    raise e

[2024-03-05 16:16:49,117: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-05 16:16:49,120: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 16:16:49,124: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-05 16:16:49,125: INFO: common: created directory at: artifacts]
[2024-03-05 16:16:49,125: INFO: common: created directory at: artifacts/model_trainer]
Learning rate set to 0.074803
0:	learn: 83.5936079	total: 60.4ms	remaining: 1m
1:	learn: 79.7934214	total: 63ms	remaining: 31.5s
2:	learn: 76.3866473	total: 65.6ms	remaining: 21.8s
3:	learn: 73.2325656	total: 67.8ms	remaining: 16.9s
4:	learn: 70.4343278	total: 70.2ms	remaining: 14s
5:	learn: 67.9606024	total: 73ms	remaining: 12.1s
6:	learn: 65.7151433	total: 75.5ms	remaining: 10.7s
7:	learn: 63.5952503	total: 78.2ms	remaining: 9.7s
8:	learn: 61.7680054	total: 80.5ms	remaining: 8.86s
9:	learn: 60.0760239	total: 82.9ms	remaining: 8.21s
10:	learn: 58.6045778	total: 85.1ms	remai